In [2]:
# import all libraries 

import requests
import lxml
import sys

# 爬虫
from bs4 import BeautifulSoup
import xlwt
import time
import urllib

import pandas as pd
import numpy

import re

In [3]:
# a helper function, 之后要用的，把中文字弄成一排 list 

def get_chinese_list(lst):
    temp = []
    for item in lst:
        temp.append(item.text+"; ")
    return (list(set(temp)))


In [4]:
count = 1


# 一定要换上你自己的 cookie 

def craw(url,key_word,x):
    global count 
    
    User_Agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'
    re = r'https://www.qcc.com/search?key='+(urllib.parse.quote(key_word))
    print ("NO.", count, "key_word", key_word)
    count+=1 
    print ("re", re)
    headers = {
            'Host':'www.qcc.com',
            'Connection': 'keep-alive',
            'Accept':r'text/html, */*; q=0.01',
            'X-Requested-With': 'XMLHttpRequest',
            'User-Agent':r'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
            'Referer': re,
            'Accept-Encoding':'gzip, deflate, br',
            'Accept-Language':'zh-CN,zh;q=0.9',
            'Cookie': r'！！！换成你自己的 cookie！！！',
            }
 
    try:
        response = requests.get(url,headers = headers)
        if response.status_code != 200:
            response.encoding = 'utf-8'
            print(response.status_code)
            print('ERROR')    
        soup = BeautifulSoup(response.text,'lxml')
    except Exception:
        print('请求都不让，这企查查是想逆天吗？？？')
    try:
        com_all_info = soup.find_all(class_='m_srchList')[0].tbody
        com_all_info_array = com_all_info.select('tr')
        
        print('开始爬取数据，请勿打开excel')
        
        for i in range(0,1):
                temp_g_name = com_all_info_array[i].select('td')[2].select('.ma_h1')[0].text    #获取公司名
                temp_g_tag = com_all_info_array[i].select('td')[2].select('.search-tags')[0].text    #获取公司标签
                temp_r_name = com_all_info_array[i].select('td')[2].select('p')[0].a.text    #获取法人名
                temp_g_money = com_all_info_array[i].select('td')[2].select('p')[0].select('span')[0].text.strip('注册资本：')    #获取注册资本
                temp_g_date = com_all_info_array[i].select('td')[2].select('p')[0].select('span')[1].text.strip('成立日期：')    #获取公司注册时间
                temp_r_email = com_all_info_array[i].select('td')[2].select('p')[1].text.split('\n')[1].strip().strip('邮箱：')    #获取法人Email
                temp_r_phone = com_all_info_array[i].select('td')[2].select('p')[1].select('.m-l')[0].text.strip('电话：')    #获取法人手机号
                temp_g_addr = com_all_info_array[i].select('td')[2].select('p')[2].text.strip().strip('地址：')    #获取公司地址
                temp_g_state = com_all_info_array[i].select('td')[3].select('.nstatus.text-success-lt.m-l-xs')[0].text.strip()  #获取公司状态
                
                website = com_all_info_array[i].find_all("a", class_='ma_h1')[0].get("href")
                website = "https://www.qcc.com/" + website
                
                print("$$$$$$$$$$$", website)
                                
                response = requests.get(website,headers = headers)
                if response.status_code != 200:
                    response.encoding = 'utf-8'
                    print(response.status_code)
                    print('ERROR')    
                soup = BeautifulSoup(response.text,'lxml')
        
                temp_g_gudong = get_chinese_list(soup.find_all("h3", class_='seo font-14'))
                
                print ("GUDONG: ", temp_g_gudong)
                
                g_name_list.append(temp_g_name)
                g_tag_list.append(temp_g_tag)
                r_name_list.append(temp_r_name)
                g_money_list.append(temp_g_money)
                g_date_list.append(temp_g_date)
                r_email_list.append(temp_r_email)
                r_phone_list.append(temp_r_phone)
                g_addr_list.append(temp_g_addr)
                g_state_list.append(temp_g_state)
                g_website_list.append(website) 
                g_gudong_list.append(temp_g_gudong)
                g_num_qcc.append(len(com_all_info_array))

    except Exception:
        no_result_list.append(key_word)
        print('好像被拒绝访问了呢...请稍后再试叭...')
         
if __name__ == '__main__':
    global g_name_list
    global g_tag_list
    global r_name_list
    global g_money_list
    global g_date_list
    global r_email_list
    global r_phone_list
    global g_addr_list
    global g_state_list
    global g_gudong_list
    global g_website_list
    global g_original_name
    global g_num_qcc
    global no_result_list
    
    g_name_list=[]
    g_tag_list=[]
    r_name_list=[]
    g_money_list=[]
    g_date_list=[]
    r_email_list=[]
    r_phone_list=[]
    g_addr_list=[]
    g_state_list=[]
    g_gudong_list = []
    g_website_list = []
    g_original_name = []
    g_num_qcc = []
    no_result_list = []

    
    num = 2 # 想检索的次数 
    sleep_time = 2.1 # 每次检索延时的秒数，不要太快，不然有可能被封号
     
    print('正在搜索，请稍后')
    
    # 我的所有数据是从一个叫 data 的文件夹里面拿出来的
    df = pd.read_excel ('data/经销商+交易对手方删了最初的.xlsx')
    
    # ’df.Client‘ 是我之前找好的，是我想要查的东西：
    company_list = df.Client
        
    for i in company_list:

        for x in range(1,num):
                            
            url = r'https://www.qcc.com/search_index?key={}&ajaxflag=1&p={}&'.format(urllib.parse.quote(i),x)
            
            # craw function 调用之前写好的程序
            s1 = craw(url,i,x)
                        
            time.sleep(sleep_time)

        
    workbook = xlwt.Workbook()
    #创建sheet对象，新建sheet
    sheet1 = workbook.add_sheet('企查查数据', cell_overwrite_ok=True)
    #---设置excel样式---
    #初始化样式
    style = xlwt.XFStyle()
    #创建字体样式
    font = xlwt.Font()
    font.name = '仿宋'
#    font.bold = True #加粗
    #设置字体
    style.font = font
    #使用样式写入数据
    print('正在存储数据，请勿打开excel')
    #向sheet中写入数据
    name_list = ['公司名字', '原本表格里面的文字', '企查查搜索结果数', '公司标签','法定法人','注册资本','成立日期','法人邮箱','法人电话','公司地址','公司状态','股东', '网站']

    
    for cc in range(0,len(name_list)):
        sheet1.write(0,cc,name_list[cc],style)
    for i in range(0,len(g_name_list)):
        print(g_name_list[i])
        sheet1.write(i+1,0,g_name_list[i],style)#公司名字
        sheet1.write(i+1,1,g_original_name[i],style)#原本表格里面的文字
        sheet1.write(i+1,2,g_num_qcc[i],style)#企查查搜索结果数
        sheet1.write(i+1,3,g_tag_list[i],style)#公司标签
        sheet1.write(i+1,4,r_name_list[i],style)#法定法人
        sheet1.write(i+1,5,g_money_list[i],style)#注册资本
        sheet1.write(i+1,6,g_date_list[i],style)#成立日期
        sheet1.write(i+1,7,r_email_list[i],style)#法人邮箱
        sheet1.write(i+1,8,r_phone_list[i],style)#法人电话
        sheet1.write(i+1,9,g_addr_list[i],style)#公司地址
        sheet1.write(i+1,10,g_state_list[i],style)#公司状态
        sheet1.write(i+1,11,g_gudong_list[i],style)#股东信息        
        sheet1.write(i+1,12,g_website_list[i],style)#网站

        
    #保存excel文件，有同名的直接覆盖
    workbook.save(r"desktop"+time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) +".xls")
    # workbook.save(r"D:\wyy-qcc-"+time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) +".xls")
    print('保存完毕~')
    print("没有搜索结果的是", no_result_list)


正在搜索，请稍后
NO. 1 key_word 七台河利丰源商贸有限公司
re https://www.qcc.com/search?key=%E4%B8%83%E5%8F%B0%E6%B2%B3%E5%88%A9%E4%B8%B0%E6%BA%90%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
请求都不让，这企查查是想逆天吗？？？
好像被拒绝访问了呢...请稍后再试叭...
NO. 2 key_word 三亚凤凰惠隆冷饮商行
re https://www.qcc.com/search?key=%E4%B8%89%E4%BA%9A%E5%87%A4%E5%87%B0%E6%83%A0%E9%9A%86%E5%86%B7%E9%A5%AE%E5%95%86%E8%A1%8C
请求都不让，这企查查是想逆天吗？？？
好像被拒绝访问了呢...请稍后再试叭...
NO. 3 key_word 三明合林贸易有限公司
re https://www.qcc.com/search?key=%E4%B8%89%E6%98%8E%E5%90%88%E6%9E%97%E8%B4%B8%E6%98%93%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8


KeyboardInterrupt: 

# 以下是 beautiful scrape 的实例

In [18]:
# 以一个静态 html 作为例子

html = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">There will have more story</p>
"""

# soup = BeautifulSoup(open('天水金河源商贸有限公司-赵卫军_企业工商信息查询-企查查.htm'))
soup = BeautifulSoup(open('阿里巴巴（中国）网络技术有限公司-戴珊_企业工商信息查询-企查查.htm'))

print (soup.prettify())


In [93]:
# 具体看怎么去爬

soup.find_all("h3", class_='seo font-14')[0]

<h3 class="seo font-14">淘宝（中国）软件有限公司</h3>

In [61]:
soup.find_all("h3", class_='seo font-14')[0].text

'淘宝（中国）软件有限公司'

In [18]:
# 爬虫的1个例子

#输出第一个 title 标签
print ("soup.title: ", soup.title)
 
#输出第一个 title 标签的标签名称
print ("soup.title.name: ", soup.title.name)
 
#输出第一个 title 标签的包含内容
print ("soup.title.string: ", soup.title.string)
 
#输出第一个 title 标签的父标签的标签名称
print ("soup.title.parent.name: ", soup.title.parent.name)
 
#输出第一个  p 标签
print ("soup.p: ", soup.p)
 
#输出第一个  p 标签的 class 属性内容
print ("soup.p['class']: ", soup.p['class'])
 
#输出第一个  a 标签的  href 属性内容
print ("soup.a['href']: ", soup.a['href'])

'''
soup的属性可以被添加,删除或修改. 再说一次, soup的属性操作方法与字典一样
'''

#修改第一个 a 标签的href属性为 http://www.baidu.com/
soup.a['href'] = 'http://www.baidu.com/'
 
#给第一个 a 标签添加 name 属性
soup.a['name'] = u'百度'
 
#删除第一个 a 标签的 class 属性为
del soup.a['class']
 
##输出第一个  p 标签的所有子节点
print (soup.p.contents)
       
#输出第一个  a 标签
print (soup.a)
 
#输出所有的  a 标签，以列表形式显示
print (soup.find_all('a'))
 
#输出第一个 id 属性等于  link3 的  a 标签
print (soup.find(id="link3"))
 
#获取所有文字内容
print(soup.get_text())
 
#输出第一个  a 标签的所有属性信息
print (soup.a.attrs)
 
for link in soup.find_all('a'):
    #获取 link 的  href 属性内容
    print(link.get('href'))
 
#对soup.p的子节点进行循环输出    
for child in soup.p.children:
    print(child)
 
#正则匹配，名字中带有b的标签
for tag in soup.find_all(re.compile("b")):
    print(tag.name)

    

soup.title:  <title>The Dormouse's story</title>
soup.title.name:  title
soup.title.string:  The Dormouse's story
soup.title.parent.name:  head
soup.p:  <p class="title" name="dromouse"><b>The Dormouse's story</b></p>
soup.p['class']:  ['title']
soup.a['href']:  http://example.com/elsie
[<b>The Dormouse's story</b>]
<a href="http://www.baidu.com/" id="link1" name="百度"><!-- Elsie --></a>
[<a href="http://www.baidu.com/" id="link1" name="百度"><!-- Elsie --></a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>
The Dormouse's story

The Dormouse's story
Once upon a time there were three little sisters; and their names were
,
Lacie and
Tillie;
and they lived at the bottom of a well.
There will have more story

{'href': 'http://www.baidu.com/', 'id': 'link1', 'name': '百度'}
http://www.baidu.com/
http://example.com/lacie
http: